# Homework - Module 02

In [1]:
import mlflow

from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

MLFLOW_TRACKING_URI ="http://127.0.0.1:5000"

## Question 1

In [2]:
print(f"MLflow Version: {mlflow.__version__}")

MLflow Version: 1.26.0


## Question 2

In [3]:
output_files = !ls homework/output
print(f"Number of files saved in output folder: {len(output_files)}")

Number of files saved in output folder: 4


## Question 3

In [4]:
from homework import train

train.run(data_path='homework/output')

2022/05/28 22:28:57 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/28 22:28:57 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/05/28 22:29:19 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/Caskroom/miniconda/base/envs/mlops/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [5]:
mlflow.list_run_infos('0')

[<RunInfo: artifact_uri='file:///Users/gpfl/scripts/python/mlops-zoomcamp/02-experiment-tracking/mlruns/0/038ce39392614243b932254ffd39cd8b/artifacts', end_time=1653787760409, experiment_id='0', lifecycle_stage='active', run_id='038ce39392614243b932254ffd39cd8b', run_uuid='038ce39392614243b932254ffd39cd8b', start_time=1653787737485, status='FINISHED', user_id='gpfl'>]

In [7]:
run = mlflow.get_run(run_id="038ce39392614243b932254ffd39cd8b") 

print(f"Number of parameters logged by MLflow: {len(run.data.params)}")

Number of parameters logged by MLflow: 17


## Question 4

In [8]:
# mlflow ui --backend-store-uri sqlite:///homework.db --default-artifact-root ./artifacts 

print("--default-artifact-root")

--default-artifact-root


## Question 5

In [25]:
%%sh
cd homework
python hpo.py

2022/05/28 20:29:56 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.


100%|██████████| 50/50 [05:56<00:00,  7.13s/trial, best loss: 6.6284257482044735]


In [9]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./artifacts/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./artifacts/2', experiment_id='2', lifecycle_stage='active', name='random-forest-hyperopt', tags={}>,
 <Experiment: artifact_location='./artifacts/3', experiment_id='3', lifecycle_stage='active', name='random-forest-best-models', tags={}>]

In [10]:
best_run = client.search_runs(
    experiment_ids=['2'], 
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=1,
    order_by=["metrics.rmse ASC"]
)

for run in best_run:
    print(f"Best RMSE: {run.data.metrics['rmse']:.3f}")

Best RMSE: 6.628


## Question 6

In [41]:
%%sh
cd homework
python register_model.py

2022/05/28 21:24:36 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
2022/05/28 21:24:36 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/28 21:24:52 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/Caskroom/miniconda/base/envs/mlops/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."
Successfully registered model 'best-random-forest'.
2022/05/28 21:26:03 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: best-random-forest, version 1
Created version '1' of model 'best-random-forest'.


In [11]:
client.list_experiments()

[<Experiment: artifact_location='./artifacts/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./artifacts/2', experiment_id='2', lifecycle_stage='active', name='random-forest-hyperopt', tags={}>,
 <Experiment: artifact_location='./artifacts/3', experiment_id='3', lifecycle_stage='active', name='random-forest-best-models', tags={}>]

In [12]:
best_run = client.search_runs(
    experiment_ids=['3'], 
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=1,
    order_by=["metrics.test_rmse ASC"]
)

print(f"Best Test RMSE: {best_run[0].data.metrics['test_rmse']:.2f}")

Best Test RMSE: 6.55
